# Exercises - Data Aquistion

All of the exercises for this module should be done within your `ds-methodologies` repository, inside of a directory named `time_series`.

The end result of this exercise should be a file named `acquire.py`.

In [1]:
from pprint import pprint
import requests
import pandas as pd
import json

1. **Using the code from the lesson as a guide, create a dataframe named `items` that has all of the data for items.**

In [2]:
base_url = 'https://python.zach.lol'

In [33]:
sect_urls = {
    'items': {'sect_key':'items', 'api_url': '/api/v1/items', 'idx_col': 'item_id', 'page_beg': 1},
    'stores': {'sect_key':'stores', 'api_url': '/api/v1/stores', 'idx_col': 'store_id', 'page_beg': 1},
    'sales': {'sect_key':'sales', 'api_url': '/api/v1/sales', 'idx_col': 'sale_id', 'page_beg': 1},
}

In [72]:
def get_json_payload_data(
    target_table='items',
    base_url='https://python.zach.lol',
    sect_url_keys=sect_urls,
    data_key='payload',
    status_key='status',
    on_page_key='page',
    of_pages_key='max_page',
    url_next_key='next_page',
    url_prev_key='previous_page',
    url_get_page='?page=',
    **kwargs):
    
    # extract sect urls keys
    url_keys = sect_url_keys[target_table]
    sect_key = url_keys['sect_key']
    api_url = url_keys['api_url']
    idx_col = url_keys['idx_col']
    beg_page=url_keys['page_beg'],
    
    # Setup df
    df = pd.DataFrame()
    
    # get initial page
    next_url = api_url
    do while next_url is not None:
        response = requests.get(base_url + next_url)

        response_json = response.json()
        payload = response_json[data_key]
        #pprint(payload)
        page_data = payload[sect_key]
        pprint(page_data)
        page_df = pd.DataFrame(page_data).set_index(idx_col)
    
    
    return df
    

In [73]:
get_json_payload_data(target_table='items', base_url=base_url)

{'items': [{'item_brand': 'Riceland',
            'item_id': 1,
            'item_name': 'Riceland American Jazmine Rice',
            'item_price': 0.84,
            'item_upc12': '35200264013',
            'item_upc14': '35200264013'},
           {'item_brand': 'Caress',
            'item_id': 2,
            'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 '
                         'Ct',
            'item_price': 6.44,
            'item_upc12': '11111065925',
            'item_upc14': '11111065925'},
           {'item_brand': 'Earths Best',
            'item_id': 3,
            'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed '
                         'Berry',
            'item_price': 2.43,
            'item_upc12': '23923330139',
            'item_upc14': '23923330139'},
           {'item_brand': 'Boars Head',
            'item_id': 4,
            'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
            'item_price': 3.14,
        

""


In [4]:
resp_items = requests.get(base_url + sect_url['items'])
json_items = resp_items.json()
# pprint(json_items)

In [5]:
pload_items = json_items['payload']
# pprint(pload_items)

In [6]:
items_df = pd.DataFrame()

In [7]:
# pprint(pload_items['page'])

In [8]:
# type(pload_items['previous_page'])

In [9]:
items_page_list = pload_items['items']
# pprint(items_page_list)

In [24]:
items_page_df = pd.DataFrame(items_page_list).set_index('item_id')

# items_page_df

In [25]:
items_df = items_df.append(items_page_df)
# items_df

In [12]:
resp_items = requests.get(base_url + pload_items['next_page'])
json_items = resp_items.json()

# pprint(json_items)

In [13]:
pload_items = json_items['payload']
# pprint(pload_items)

In [14]:
items_page_list = pload_items['items']
# pprint(items_page_list)

In [15]:
items_page_df = pd.DataFrame(items_page_list).set_index('item_id')

# items_page_df

In [16]:
items_df = items_df.append(items_page_df, verify_integrity=True)

# items_df

In [17]:
resp_items = requests.get(base_url + pload_items['next_page'])
json_items = resp_items.json()

# pprint(json_items)

In [18]:
pload_items = json_items['payload']
# pprint(pload_items)

In [19]:
items_page_list = pload_items['items']
# pprint(items_page_list)

In [20]:
items_page_df = pd.DataFrame(items_page_list).set_index('item_id')

# items_page_df

,item_brand,item_name,item_price,item_upc12,item_upc14
item_id,,,,,
41,P.f. Changs,P.f. Changs Home Menu Meal For Two Beef With B...,5.62,31000670016,31000670016
42,Moms Best Naturals,Moms Best Naturals Cereal Toasted Cinnamon Squ...,2.97,883978129115,883978129115
43,Ferrara,Ferrara Vanilla Syrup,8.40,71403000379,71403000379
44,Elmers,Elmers Board Mate Dual Tip Glue Pen,7.06,26000001403,26000001403
45,Kelloggs,Kelloggs Disney Pixar Cars 2 Cereal,4.40,38000542237,38000542237
46,Mama Marys,Pizza Sauce,4.65,35457770664,35457770664
47,Bear Naked,Bear Naked Fit Almond Crisp 100 Percent Natura...,7.38,884623708976,884623708976
48,Dove,Dove Men + Care Antiperspirant Deodorant Cool ...,3.72,79400271631,79400271631
49,Easy-off,Easy-off Oven Cleaner Lemon Scent,9.54,62338879772,62338879772


In [21]:
items_df = items_df.append(items_page_df, verify_integrity=True)

items_df

,item_brand,item_name,item_price,item_upc12,item_upc14
item_id,,,,,
1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,Caress,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
3,Earths Best,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
4,Boars Head,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
5,Back To Nature,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
6,Sally Hansen,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
7,Twinings Of London,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
8,Lea & Perrins,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
9,Van De Kamps,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015


2. **Do the same thing, but for `stores`.**

In [22]:
resp_items = requests.get(base_url + sect_url['stores'])
json_items = resp_items.json()
pprint(json_items)

{'payload': {'max_page': 1,
             'next_page': None,
             'page': 1,
             'previous_page': None,
             'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
                         'store_city': 'San Antonio',
                         'store_id': 1,
                         'store_state': 'TX',
                         'store_zipcode': '78253'},
                        {'store_address': '9255 FM 471 West',
                         'store_city': 'San Antonio',
                         'store_id': 2,
                         'store_state': 'TX',
                         'store_zipcode': '78251'},
                        {'store_address': '2118 Fredericksburg Rdj',
                         'store_city': 'San Antonio',
                         'store_id': 3,
                         'store_state': 'TX',
                         'store_zipcode': '78201'},
                        {'store_address': '516 S Flores St',
                         'store_city': 'San 

3. **Extract the data for `sales`. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.**

In [23]:
resp_items = requests.get(base_url + sect_url['sales'])
json_items = resp_items.json()
pprint(json_items)

{'payload': {'max_page': 183,
             'next_page': '/api/v1/sales?page=2',
             'page': 1,
             'previous_page': None,
             'sales': [{'item': 1,
                        'sale_amount': 13.0,
                        'sale_date': 'Tue, 01 Jan 2013 00:00:00 GMT',
                        'sale_id': 1,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 11.0,
                        'sale_date': 'Wed, 02 Jan 2013 00:00:00 GMT',
                        'sale_id': 2,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 14.0,
                        'sale_date': 'Thu, 03 Jan 2013 00:00:00 GMT',
                        'sale_id': 3,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 13.0,
                        'sale_date': 'Fri, 04 Jan 2013 00:00:00 GMT',
                        'sale_id': 

                       {'item': 1,
                        'sale_amount': 24.0,
                        'sale_date': 'Wed, 03 Apr 2013 00:00:00 GMT',
                        'sale_id': 93,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 18.0,
                        'sale_date': 'Thu, 04 Apr 2013 00:00:00 GMT',
                        'sale_id': 94,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 19.0,
                        'sale_date': 'Fri, 05 Apr 2013 00:00:00 GMT',
                        'sale_id': 95,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 23.0,
                        'sale_date': 'Sat, 06 Apr 2013 00:00:00 GMT',
                        'sale_id': 96,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 17.0,
                

                        'store': 1},
                       {'item': 1,
                        'sale_amount': 20.0,
                        'sale_date': 'Thu, 10 Oct 2013 00:00:00 GMT',
                        'sale_id': 283,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 17.0,
                        'sale_date': 'Fri, 11 Oct 2013 00:00:00 GMT',
                        'sale_id': 284,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 17.0,
                        'sale_date': 'Sat, 12 Oct 2013 00:00:00 GMT',
                        'sale_id': 285,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 20.0,
                        'sale_date': 'Sun, 13 Oct 2013 00:00:00 GMT',
                        'sale_id': 286,
                        'store': 1},
                       {'item': 1,
                    

                       {'item': 1,
                        'sale_amount': 20.0,
                        'sale_date': 'Sun, 27 Apr 2014 00:00:00 GMT',
                        'sale_id': 482,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 15.0,
                        'sale_date': 'Mon, 28 Apr 2014 00:00:00 GMT',
                        'sale_id': 483,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 18.0,
                        'sale_date': 'Tue, 29 Apr 2014 00:00:00 GMT',
                        'sale_id': 484,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 13.0,
                        'sale_date': 'Wed, 30 Apr 2014 00:00:00 GMT',
                        'sale_id': 485,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 22.0,
            

                        'sale_id': 649,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 29.0,
                        'sale_date': 'Sun, 12 Oct 2014 00:00:00 GMT',
                        'sale_id': 650,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 6.0,
                        'sale_date': 'Mon, 13 Oct 2014 00:00:00 GMT',
                        'sale_id': 651,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 20.0,
                        'sale_date': 'Tue, 14 Oct 2014 00:00:00 GMT',
                        'sale_id': 652,
                        'store': 1},
                       {'item': 1,
                        'sale_amount': 21.0,
                        'sale_date': 'Wed, 15 Oct 2014 00:00:00 GMT',
                        'sale_id': 653,
                        'store': 1},
                

...

                        'sale_date': 'Mon, 02 Jun 2014 00:00:00 GMT',
                        'sale_id': 4170,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 27.0,
                        'sale_date': 'Tue, 03 Jun 2014 00:00:00 GMT',
                        'sale_id': 4171,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 22.0,
                        'sale_date': 'Wed, 04 Jun 2014 00:00:00 GMT',
                        'sale_id': 4172,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 22.0,
                        'sale_date': 'Thu, 05 Jun 2014 00:00:00 GMT',
                        'sale_id': 4173,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 22.0,
                        'sale_date': 'Fri, 06 Jun 2014 00:00:00 GMT',
                  

                       {'item': 1,
                        'sale_amount': 15.0,
                        'sale_date': 'Thu, 01 Jan 2015 00:00:00 GMT',
                        'sale_id': 4383,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 16.0,
                        'sale_date': 'Fri, 02 Jan 2015 00:00:00 GMT',
                        'sale_id': 4384,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 20.0,
                        'sale_date': 'Sat, 03 Jan 2015 00:00:00 GMT',
                        'sale_id': 4385,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 18.0,
                        'sale_date': 'Sun, 04 Jan 2015 00:00:00 GMT',
                        'sale_id': 4386,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 8.0,
         

                        'store': 3},
                       {'item': 1,
                        'sale_amount': 28.0,
                        'sale_date': 'Wed, 03 Jun 2015 00:00:00 GMT',
                        'sale_id': 4536,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 36.0,
                        'sale_date': 'Thu, 04 Jun 2015 00:00:00 GMT',
                        'sale_id': 4537,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 31.0,
                        'sale_date': 'Fri, 05 Jun 2015 00:00:00 GMT',
                        'sale_id': 4538,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 39.0,
                        'sale_date': 'Sat, 06 Jun 2015 00:00:00 GMT',
                        'sale_id': 4539,
                        'store': 3},
                       {'item': 1,
                

                        'store': 3},
                       {'item': 1,
                        'sale_amount': 15.0,
                        'sale_date': 'Mon, 28 Dec 2015 00:00:00 GMT',
                        'sale_id': 4744,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 21.0,
                        'sale_date': 'Tue, 29 Dec 2015 00:00:00 GMT',
                        'sale_id': 4745,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 16.0,
                        'sale_date': 'Wed, 30 Dec 2015 00:00:00 GMT',
                        'sale_id': 4746,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 23.0,
                        'sale_date': 'Thu, 31 Dec 2015 00:00:00 GMT',
                        'sale_id': 4747,
                        'store': 3},
                       {'item': 1,
                

                        'sale_id': 4976,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 33.0,
                        'sale_date': 'Wed, 17 Aug 2016 00:00:00 GMT',
                        'sale_id': 4977,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 37.0,
                        'sale_date': 'Thu, 18 Aug 2016 00:00:00 GMT',
                        'sale_id': 4978,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 26.0,
                        'sale_date': 'Fri, 19 Aug 2016 00:00:00 GMT',
                        'sale_id': 4979,
                        'store': 3},
                       {'item': 1,
                        'sale_amount': 36.0,
                        'sale_date': 'Sat, 20 Aug 2016 00:00:00 GMT',
                        'sale_id': 4980,
                        'store': 3},
          

4. **Save the data in your files to local csv files so that it will be faster to access in the future.**

5. **Combine the data from your three separate dataframes into one large dataframe.**

6. **Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here:** https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

7. **Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the `acquire.py` file and be able to re-run the functions and get the same data.**